In [26]:
import os, time, shutil
import numpy as np

In [24]:
import spikeinterface.full as si


In [12]:
input_path = '/net/bs-filesvr02/export/group/hierlemann/recordings/Mea1k/phornauer/Optogenetics/'
save_path = '/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Mea1k/phornauer/si_test/'
rec_date = '220628'
chip_id = 'M04256'
run_id = '000063'
file_name = "data.raw.h5"

In [11]:
rec_path = os.path.join(input_path,rec_date,chip_id,'Network',)
assert(os.path.exists(rec_path))

In [17]:
si.sorters.available_sorters()

['combinato',
 'hdsort',
 'herdingspikes',
 'ironclust',
 'kilosort',
 'kilosort2',
 'kilosort2_5',
 'kilosort3',
 'klusta',
 'mountainsort4',
 'pykilosort',
 'spykingcircus',
 'spykingcircus2',
 'tridesclous',
 'tridesclous2',
 'waveclus',
 'waveclus_snippets',
 'yass']

In [25]:
si.kilosort2.Kilosort2Sorter.set_kilosort2_path('/home/phornauer/Git/Kilosort2/')

Setting KILOSORT2_PATH environment variable for subprocess calls to: /home/phornauer/Git/Kilosort2


In [27]:
### sorter name ###
sorter = 'kilosort2'
sorter = 'tridesclous2'

### sorter params - only params of the sorter used ###
sorter_params = {"n_jobs_bin": 8, "total_memory": "8G", "NT": 1*1024+64}

In [28]:
### Set parameters ###
# If true, and spike sorting output is present, it's deleted and resorted
recompute_sorting = False
recompute_curation = False
# If true, filtered data and sorted outputs are saved in a format that it's easy to retrieve (.pkl)
dump_recording = True
dump_sorting = True
# If true, exports to Phy
export_raw_to_phy = False
export_curated_to_phy = False
# If true, unit templates are plotted for all units
plot_unit_templates = True
plot_image = True
### Filter params ###
freq_min = 150
freq_max = 3000
### Automatic curation ###
# If true, output is automatically curated using quality metrics (QC)
auto_curate = True
# Thresholds for automatic curations (if any is None, that autocuration is skipped
# ISI-violation ratio (greater values are removed)
isi_viol_thresh = 0.5
# firing rate (smaller values are removed)
fr_thresh = 0.05
# signal-to-noise ratio (smaller values are removed)
snr_thresh = 5
### Other processing params - used for all sorters ###
# number of jobs to use
n_jobs = 8
# total RAM to use
total_memory = "500M"
# chunk size
chunk_size = 10000
# Number of spikes per unit to compute templates (None-> all spikes are used)
max_spikes_per_unit = None
# Number of channels to compute center of mass
num_channels_for_com = 30

In [29]:

for i in range(0): #,5,1):
    output_folder = save_path / f"well0{i}" / 'sorted'
    cache_folder = save_path / f"well0{i}" / 'cache'
    figures_folder = save_path / f"well0{i}" / 'figures'
    output_folder.mkdir(parents=True, exist_ok=True)
    cache_folder.mkdir(parents=True, exist_ok=True)
    figures_folder.mkdir(parents=True, exist_ok=True)
    tmp_folder = cache_folder / 'tmp' / sorter
    tmp_folder.mkdir(parents=True, exist_ok=True)
    # Load recording
    try:
        print("Trying to load Maxwell recording")
        rec = si.MaxwellRecordingExtractor(rec_path, stream_id=f"well00{i}")
    except:
        raise Exception(f"Could not open the provided file: {rec_path} with the MaxwellRecordingExtractor")

    print(f"DURATION: {rec.get_num_frames() / rec.get_sampling_frequency()} s -- "
          f"NUM. CHANNELS: {rec.get_num_channels()}")
    print(rec.get_sampling_frequency())

    ### Filter and dumping
    if (cache_folder / 'recording').is_dir():
        print("Loading saved recording")
        rec_cache = si.load_extractor(cache_folder / 'recording')
    else:
        print('FILTERING\n')
        rec_f = si.preprocessing.bandpass_filter(rec, freq_min=freq_min, freq_max=freq_max)

        if dump_recording:
            start = time.time()
            rec_cache = rec_f.save(folder=cache_folder / "recording", n_jobs=n_jobs, chunk_size=chunk_size,
                                   progress_bar=True)
            stop = time.time()
            print(f'Elapsed saving time {np.round(stop - start, 2)}\n')
            print(f"Filtered recording saved to {cache_folder / 'recording'}\n")
        else:
            rec_cache = rec_f
    fs = rec_cache.get_sampling_frequency()

    ### Spike sorting
    if recompute_sorting and output_folder.is_dir():
        shutil.rmtree(output_folder)
    try:
        if not (cache_folder / 'sorting_raw').is_dir():
            print(f'SORTING WITH {sorter}\n')
            t_start_sort = time.time()
            sorting = si.run_sorter(sorter, rec_cache, output_folder=output_folder, verbose=True,
                                    **sorter_params)
            print(f"\n\nSpike sorting elapsed time {time.time() - t_start_sort} s")
        else:
            print('Skipping', rec_path, ' since already sorted')
            sorting = si.load_extractor(cache_folder / 'sorting_raw')
    except Exception as e:
        print(f"{sorter} failed on recording {rec_path}\n\nError: {e}")
        continue

    if export_raw_to_phy and not (cache_folder / "phy_raw").is_dir():
        we_raw = si.extract_waveforms(rec_cache, sorting, folder=cache_folder / "waveforms_raw", load_if_exists=True,
                                      n_jobs=n_jobs, total_memory=total_memory, progress_bar=True)
        print("Exporting raw sorting output to Phy")
        si.export_to_phy(we_raw, cache_folder / "phy_raw", n_jobs=n_jobs, total_memory=total_memory,
                      progress_bar=True)


'well05'